In [15]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math
import scipy.integrate as si
import scipy.optimize as so

In [16]:
# PARAMeTERS

incremental_Efrac = 7.8 # Choose from the following values: 0.1, 7.8
save_to_disk = True

In [17]:
def second_derivative_parametric_input(x, y, t):
    # Input should be 'x' in terms 't', and 'y' in terms of 't'
    # Output is second derivative of 'y' with respect to 'x' ('d2y/dx2') as a function of 't'
    dy_dx = sp.Derivative(y, t) / sp.Derivative(x, t)
    d2y_dx2 =  sp.Derivative(dy_dx, t) / sp.Derivative(x, t)
    d2y_dx2_in_terms_of_t = sp.nsimplify(d2y_dx2.doit())


    return d2y_dx2_in_terms_of_t
    

In [18]:
c, f_NFW, x_prime = sp.symbols("c f_NFW x_prime", positive=True)

# For NFW AMC

crho_NFW = 1 / (c*x_prime) / (1+c*x_prime)**2 # dimensionless density of NFW profile. 'chro_NFW = rho_NFW / rho_s', where 'rho_NFW' is the density profile and 'rho_s' is the scale radius.
psi_2_NFW = 1/f_NFW * ( sp.log(1+c*x_prime)/x_prime - c/(1+c*x_prime) )  # normalized relative potential. From equation (25) in our article

d2crho_dpsi2_NFW = second_derivative_parametric_input(psi_2_NFW, crho_NFW, x_prime) # Second derivative of 'chro_NFW' with respect to 'psi_2_NFW' as a functin of 'x_prime'


# For Hernquist AMC

crho_Hern = 1 / (c*x_prime) / (1+c*x_prime)**3 # dimensionless density of Hernquist profile. 'chro_Hern = rho_Hern / rho_1', where 'rho_Hern' is the density profile and 'rho_1' is the scale radius.
psi_2_Hern = (1+c)**2 * x_prime / (1+c*x_prime)**2 # normalized relative potential. From equation (30) in our article

d2crho_dpsi2_Hern = second_derivative_parametric_input(psi_2_Hern, crho_Hern, x_prime) # Second derivative of 'chro_Hern' with respect to 'psi_2_Hern' as a functin of 'x_prime'



In [19]:
# NFW functions

# compute normalized crossover radius 'x*'
def get_x_star_NFW():
    eqn = lambda x_c: beta / alpha_squared * E_frac * x_c**2 - ( 1/f_NFW_val * ( np.log(1+c_val*x_c) / x_c  -  c_val / (1+c_val*x_c) ) )
    sol = so.fsolve(eqn, 1)[0]
    
    return sol

# Computes normalized radius 'x' given a value of normalized relative potential 'psi_B', using equation (25) in our article
def get_x_prime_NFW(psi_prime):
    eqn = lambda x_Prime: psi_prime  -  1/f_NFW_val * ( np.log(1+c_val*x_Prime)/x_Prime - c_val/(1+c_val*x_Prime) )
    #sol = so.fsolve(eqn, 1)[0]
    sol = so.root_scalar(eqn, bracket=(x_star, 1e20)).root
    
    return sol

# Compute the integrand according to equations (J8) or (23) in our article
def integrand_1_NFW(psi_prime, eps, x):
    
    psi_1_NFW = 1/f_NFW_val * ( math.log(1+c_val*x)/x - c_val/(1+c_val*x_star) )
    x_prime = get_x_prime_NFW(psi_prime)

    return (
        x**2 * 1/(math.sqrt(8)*math.pi**2) * 1/np.sqrt(eps - psi_prime) * 
        d2crho_dpsi2_c_and_fNFW_substituted_lambdified_NFW(x_prime).real * np.sqrt(2*(psi_1_NFW - eps))
    )

# compute the integrand according to equation (21) in our article
def integrand_2_NFW(x):
    return x / (1 + c_val*x)**2

def range_x_1_NFW():
    return [0, min(x_star, 1)]

def range_x_2_NFW():
    return [0, min(x_star, 1)]

def range_x_1_NFW_I():
    return [0, x_star]


# Hernquist functions

# compute normalized crossover radius 'x*'
def get_x_star_Hern():
    eqn = lambda x_c: beta / alpha_squared * E_frac * x_c**2 -  (1+c_val)**2 * x_c / (1+c_val*x_c)**2 
    sol = so.fsolve(eqn, 1)[0]
    
    return sol

# Computes normalized radius 'x' given a value of normalized relative potential 'psi_B', using equation (30) in our article
def get_x_prime_Hern(psi_prime):
    eqn = lambda x_Prime: psi_prime  -  (1+c_val)**2 * x_Prime / (1+c_val*x_Prime)**2
    sol = so.root_scalar(eqn, bracket=(x_star, 1e20)).root
    
    return sol

# Compute the integrand according to equations (65) in our article
def integrand_1_Hern(psi_prime, eps, x):
    
    psi_1_Hern = (1+c_val)**2 * ( (x_star - x)/(1 + c_val*x_star)/(1 + c_val*x) + x_star/(1+c_val*x_star)**2 ) # uses equation (29) in our article
    x_prime = get_x_prime_Hern(psi_prime)

    return (
        1/(math.sqrt(8)*math.pi**2) * x**2 * np.sqrt(2*(psi_1_Hern - eps)) * 1/np.sqrt(eps - psi_prime) * 
        d2crho_dpsi2_c_substituted_lambdified_Hern(x_prime)
    )


def integrand_2_Hern(x):
    return x / (1 + c_val*x)**3

# compute conentration of Hernquist profile given its scale density. Using equation (55) in our article
def compute_c_Hern(rho):
    eqn = lambda c_Hern: 1 / 2 / c_Hern / (1 + c_Hern)**2   -   200 / 3 * rho_crit / rho
    sol = so.fsolve(eqn, 1)[0]
    return sol

def range_x_2_Hern():
    x_i_NFW = cs[0] / c_val /  R_factor
    return [0, min(x_star, x_i_NFW)]


def range_x_1_Hern():
    x_i_NFW = cs[0] / c_val /  R_factor
    return [0, min(x_star, x_i_NFW)]

def range_x_1_Hern_I():
    return [0, x_star]



# Functions common for both NFW and Hernquist AMCs. Therefore, no suffix.

def range_eps_1(x):
    return [0, beta / alpha_squared * E_frac * x**2]

def range_psi_prime_1(eps, x):
    return [0, eps]



#------------------------------------------------------------------------




rho_crit = 9.1275e-27 # in kg/m^3. Cosmological critical density 

# Set up the incremental energy injection parameters 'E_fracs' corresponding to successive stellar encounters 
if incremental_Efrac == 0.1:
    E_fracs = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2]
elif incremental_Efrac == 7.8:
    E_fracs = [7.8, 7.8, 7.8, 7.8, 7.8]
else:
    raise ValueError("Invalid incremental E_frac entered. Set 'incremental_Efrac' to one of the following values: 0.1, 7.8")


cs = np.zeros(np.size(E_fracs) + 1) #1st one is for the NFW AMC. The remaining ones are for the successive generations of Hernquist AMCs
cs[0] = 100      #Initializing the concentration of the unperturbed NFW AMC
f_NFW_val = math.log(1+cs[0])-cs[0]/(1+cs[0]) # using equation (14) in our article

# Scale densities of AMCs
rhos = np.zeros(np.size(cs))
rhos[0] = 200/3 * cs[0]**3/(np.log(1+cs[0]) - cs[0]/(1+cs[0])) * rho_crit # Setting the scale density of the unperturbed NFW minihalo. Uses equation (49) in our article

x_stars = np.zeros(np.size(E_fracs))

survival_fractions = np.zeros(np.size(E_fracs)) # Create a zeros array the same size as the 'E_fracs' array. This will eventually contain the values of survival fractions corresponding to E_frac values.

Rs = np.zeros(np.size(cs) - 1) # We need one less than the total number AMCs in the code. These are the ratio of scale radius of child minihalo to that of parent minihalo.

for i in range(np.size(E_fracs)):

    c_val = cs[i]
    E_frac = E_fracs[i]

    if i == 0: # parent is NFW minihalo
        alpha_squared = 3/c_val**2 + 1/(2*f_NFW_val)*(c_val-3)/(c_val+1)
        beta = (c_val**3 - 2*c_val*(1+c_val)*f_NFW_val) / (2*(1+c_val)**2 * f_NFW_val**2) # this is actually gamma in our article

        d2crho_dpsi2_c_and_fNFW_substituted_NFW = d2crho_dpsi2_NFW.subs({c:c_val, f_NFW:f_NFW_val})  # substitute the numerical values of 'concentration' and 'f_NFW' in 'd2crho_dpsi2_NFW'
        d2crho_dpsi2_c_and_fNFW_substituted_lambdified_NFW = sp.lambdify(x_prime, d2crho_dpsi2_c_and_fNFW_substituted_NFW) # Lambdify 'd2crho_dpsi2_c_and_fNFW_substituted_NFW' so that you can evaluate it by passing a numerical
                                                                                                                           # value of 'x_prime' as input

        # Compute the normalized crossover radius 'x*'
        x_star = get_x_star_NFW()
        x_stars[i] = x_star

    # --------------------------------------------------------------------------

        # Method 1

        # Compute mass of 0 < x < x_star region assuming no mass loss in this region - computes the survival fraction equivalent
        mass_fraction = c_val**2/f_NFW_val * si.nquad(integrand_2_NFW, [range_x_2_NFW])[0]
        # Compute partial mass loss in 0 < x < x_star region - computes the survival fraction equivalent
        partial_mass_loss_fraction = 4*math.pi*c_val**3/f_NFW_val * si.nquad(integrand_1_NFW, [range_psi_prime_1, range_eps_1, range_x_1_NFW])[0]

        survival_fraction_1 = mass_fraction - partial_mass_loss_fraction # uses equation (20) in our article


    # --------------------------------------------------------------------------

        # Method 2
        
    
        # Compute concentration of first Hernquist AMC
        K = np.log(1 + c_val*x_star) - c_val*x_star/(1 + c_val*x_star) # this is 'f_NFW(c_s * x_s*)'
        I = si.nquad(integrand_1_NFW, [range_psi_prime_1, range_eps_1, range_x_1_NFW_I])[0] # from equation (J8) in our article
        Rs[i] = np.sqrt(2*K - 8*np.pi*c_val**3*I) #from equation (45) in our article
        rhos[i+1] = rhos[i] / Rs[i] # from equation (42) in our article
        cs[i+1] = compute_c_Hern(rhos[i+1])  # compute the concentratin of the Hernquist profile given its scale density

        x_1_rVirS = c_val/cs[i+1] / Rs[i] # equation (K3) in our article
        H = (cs[i+1]*x_1_rVirS)**2 / (1 + cs[i+1]*x_1_rVirS)**2 # this is just 'f_Hern(c_1*x_1_rVirS)'
        survival_fraction_2 = 1/2 * Rs[i]**2 * H/f_NFW_val # equation (K9) in our article

    # --------------------------------------------------------------------------

        survival_fractions[i] = min(survival_fraction_1, survival_fraction_2) # this is the crux of the swtiching procedure


    else: # parent is Hernquist minihalo
        alpha_squared = ( c_val*(6 + 9*c_val + 2*c_val**2) - 6*(1 + c_val)**2*math.log(1+c_val) ) / c_val**4 # uses equation (31) in our article
        beta = (4 + c_val) / 6 # uses equation (32) in our article

        d2crho_dpsi2_c_substituted_Hern = d2crho_dpsi2_Hern.subs({c:c_val})  # substitute the numerical value of 'concentration' and in 'd2crho_dpsi2_Hern'
        d2crho_dpsi2_c_substituted_lambdified_Hern = sp.lambdify(x_prime, d2crho_dpsi2_c_substituted_Hern) # Lambdify 'd2crho_dpsi2_c_substituted_Hern' so that you can evaluate it by passing a numerical
                                                                                                           # value of 'x_prime' as input
        
        # Compute the normalized crossover radius 'x*'
        x_star = get_x_star_Hern()
        x_stars[i] = x_star
        

        R_factor = 1 # Will eventualy contain: Product of R_(i-1) * R_(i-2) * ... * R_1 * R_s
        for j in range(i):
            R_factor = R_factor * Rs[j]
        rn_by_rs = R_factor

    # --------------------------------------------------------------------------
        # Not doin the Method 1 because a Hernquist AMC always relaxes to a Hernquist AMC
        # Method 1

        # term1 = rn_by_rs**2 * c_val**2/f_NFW_val * si.nquad(integrand_2_Hern, [range_x_2_Hern])[0]
        # term2 = 4*np.pi * rn_by_rs**2 * c_val**3/f_NFW_val * si.nquad(integrand_1_Hern, [range_psi_prime_1, range_eps_1, range_x_1_Hern])[0]
        # survival_fraction_1 = term1 - term2

    # --------------------------------------------------------------------------

        # Method 2

        # Compute concentration of (i+1)th Hernquist AMC
        K = c_val**2*x_star**2 / (1 + c_val*x_star)**2 # this is 'f_Hern(c_n * x_n*)'
        I = si.nquad(integrand_1_Hern, [range_psi_prime_1, range_eps_1, range_x_1_Hern_I])[0] # uses equation (65) in our article
        Rs[i] = np.sqrt(K - 8*np.pi*c_val**3*I) # Uses equation (64) in our article
        rhos[i+1] = rhos[i] / Rs[i] # uses equation (63) in our article
        cs[i+1] = compute_c_Hern(rhos[i+1]) 

        x_iPlus1_rVirS = cs[0]/cs[i+1] * 1/(Rs[i] * rn_by_rs) # Product of R_(i) * R_(i-1) * ... * R_1 * R_s. Uses equation (69) in our article
        H = (cs[i+1]*x_iPlus1_rVirS)**2 / (1 + cs[i+1]*x_iPlus1_rVirS)**2 # this is just 'f_Hern(c_(n+1) * x_(n+1)^rvirs)'
        survival_fraction_2 = 1/2 * (Rs[i] * rn_by_rs)**2 * H/f_NFW_val   # Product of R_(i) * R_(i-1) * ... * R_1 * R_s # Uses equation (68) in our article

    # --------------------------------------------------------------------------    
        
        survival_fractions[i] = survival_fraction_2


        
        

    print(f"Iteration #{i+1}")



C:\Users\ids29\AppData\Local\Temp\ipykernel_8860\3628105810.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eqn = lambda x_c: beta / alpha_squared * E_frac * x_c**2 - ( 1/f_NFW_val * ( math.log(1+c_val*x_c) / x_c  -  c_val / (1+c_val*x_c) ) )


Iteration #1
Iteration #2
Iteration #3
Iteration #4
Iteration #5


In [20]:
# Save survival fraction values to disk
if save_to_disk:
    if incremental_Efrac == 0.1:
        np.save("survival_fractions_Efrac_zeroPoint1_multiple_encounters_fixed_Efrac_switching", survival_fractions)
    elif incremental_Efrac == 7.8:
        np.save("survival_fractions_Efrac_7Point8_multiple_encounters_fixed_Efrac_switching", survival_fractions)